In [2]:
import requests
from fake_useragent import UserAgent
from IPython.display import display, HTML
import pandas as pd
import scrapy
from bs4 import BeautifulSoup
from requests_html import HTMLSession, AsyncHTMLSession
import nest_asyncio

In [45]:
def get_info(query, page_num):
    url = f'https://www.sainsburys.co.uk/groceries-api/gol-services/product/v1/product?filter[keyword]={query}&include[PRODUCT_AD]=citrus&citrus_max_number_ads=5&page_number={page_num}&page_size=60&sort_order=FAVOURITES_FIRST&salesWindow=1'
    response = requests.get(url, headers=headers)
    data = response.json()
    
    if not data['products']:
        return pd.DataFrame()
    
    products = []
    for product in data['products']:
        products.append({
            'name': product['name'],
            'price': product['retail_price']['price'],
            'link': product['full_url']
        })
    
    return pd.DataFrame(products)

In [46]:
ua = UserAgent()
headers = {'User-Agent': ua.random} #randomly generated user agent


if ' ' in query:
    query = query.replace(' ', '_')

df = pd.DataFrame(columns=['name', 'price', 'link'])
for page in range(1, 100):
    temp_df = get_info("banana", page)
    if temp_df.empty:
        break
    else:
        df = pd.concat([df, temp_df], ignore_index=True)

df


/var/folders/zz/k81wn_s93cdffqmg3682c6dc0000gn/T/ipykernel_72259/2238383281.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


,name,price,link
0,Sainsbury's Fairtrade Bananas Loose,0.90,https://www.sainsburys.co.uk/shop/gb/groceries...
1,Sainsbury's Fairtrade Bananas x5,0.78,https://www.sainsburys.co.uk/shop/gb/groceries...
2,Sainsbury's Fairtrade Bananas x8,1.25,https://www.sainsburys.co.uk/shop/gb/groceries...
3,"Sainsbury's Small Bananas, Fairtrade x8",1.00,https://www.sainsburys.co.uk/shop/gb/groceries...
4,"Sainsbury's Fairtrade Bananas, SO Organic x6",1.80,https://www.sainsburys.co.uk/shop/gb/groceries...
...,...,...,...
140,Sainsbury's Glace Cherries 200g,1.80,https://www.sainsburys.co.uk/shop/gb/groceries...
141,Sainsbury's Chopped Mixed Nuts 200g,1.75,https://www.sainsburys.co.uk/shop/gb/groceries...
142,Askeys Treat Chocolate Dessert Sauce 325g,1.75,https://www.sainsburys.co.uk/shop/gb/groceries...
143,Sainsbury's Almonds Chopped 100g,1.70,https://www.sainsburys.co.uk/shop/gb/groceries...


## need data from individual product pages but selenium is going to take too much time and beauitful soup can't access dynamic content?? - need to use something else. i think scrapy might work - trying requests first

In [ ]:
session = HTMLSession()
r = session.get("https://www.sainsburys.co.uk/gol-ui/product/sainsburys-loose-fairtrade-bananas")

soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<html>
 <head>
  <title>
   Access Denied
  </title>
 </head>
 <body>
  <h1>
   Access Denied
  </h1>
  You don't have permission to access "http://www.sainsburys.co.uk/gol-ui/product/sainsburys-loose-fairtrade-bananas" on this server.
  <p>
   Reference #18.263f655f.1749225903.9b8eead
   <p>
    https://errors.edgesuite.net/18.263f655f.1749225903.9b8eead
   </p>
  </p>
 </body>
</html>



In [66]:
nest_asyncio.apply()

product_url = "https://www.sainsburys.co.uk/gol-ui/product/sainsburys-loose-fairtrade-bananas"
asession = AsyncHTMLSession()

async def scrape_product_page(url):
    response = await asession.get(url)
    await response.html.arender()
    return response.html.html

html_content = await scrape_product_page(product_url)
print(html_content)
display(HTML(html_content))

# so since nutrition values are in a dynamic dropdown, we need to scrape the page with something else

<html><head>
<title>Access Denied</title>
</head><body>
<h1>Access Denied</h1>
 
You don't have permission to access "http://www.sainsburys.co.uk/gol-ui/product/sainsburys-loose-fairtrade-bananas" on this server.<p>
Reference #18.acc2645f.1749227622.8c1dc3e
</p><p>https://errors.edgesuite.net/18.acc2645f.1749227622.8c1dc3e</p>


</body></html>


## wow, no hidden api and sainsbury blocks all bots, selenium gonna take too much time and can't access denied with requests

## failed asda attempt below :(

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache"
}

session = requests.Session()
response = session.get(
    "https://groceries.asda.com/product/bananas/asda-organic-5-fairtrade-bananas/1000128213094",
    headers=headers,
    timeout=20
)

soup = BeautifulSoup(response.content, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html data-appversion="v179" data-local-env="" lang="en">
 <head>
  <script>
   (function(){ window.SS = window.SS || {}; SS.Require = function (callback){ if (typeof callback === 'function') { if (window.SS && SS.EventTrack) { callback(); } else { var siteSpect = document.getElementById('siteSpectLibraries'); var head = document.getElementsByTagName('head')[0]; if (siteSpect === null && typeof head !== 'undefined') { siteSpect = document.createElement('script'); siteSpect.type = 'text/javascript'; siteSpect.src = '/__ssobj/core.js+ssdomvar.js+generic-adapter.js';siteSpect.async = true; siteSpect.id = 'siteSpectLibraries'; head.appendChild(siteSpect); } if (window.addEventListener){ siteSpect.addEventListener('load', callback, false); } else { siteSpect.attachEvent('onload', callback, false); } } } };})();
  </script>
  <script>
   try {

            // Response Interceptor
            if(window.fetch){
                const { fetch: originalFetch } = window;
          